In [ ]:
import numpy as np
from collections import defaultdict, Counter
import math

# -------------------------
# (unchanged) Load corpus
# -------------------------
def load_corpus(filepath):
    with open(filepath, 'r') as f:
        words = [line.strip().upper() for line in f if line.strip()]
    return words

# -------------------------
# Train HMM (bigram Markov) on words for a given length
# returns: start_prob (dict), transition_prob (dict of dicts)
# -------------------------
def train_hmm_for_words(words, alphabet=None, smoothing=1.0):
    if alphabet is None:
        all_letters = [chr(ord('A') + i) for i in range(26)]
    else:
        all_letters = alphabet

    transitions = defaultdict(Counter)
    start_letter_counter = Counter()

    for word in words:
        if len(word) == 0:
            continue
        start_letter_counter[word[0]] += 1
        for i in range(len(word) - 1):
            current_letter = word[i]
            next_letter = word[i + 1]
            transitions[current_letter][next_letter] += 1

    # start probs with add-k smoothing
    total_starts = sum(start_letter_counter.values())
    V = len(all_letters)
    start_prob = {letter: (start_letter_counter[letter] + smoothing) / (total_starts + smoothing * V) for letter in all_letters}

    transition_prob = {}
    for letter in all_letters:
        next_counts = transitions[letter]
        total_next = sum(next_counts.values())
        transition_prob[letter] = {
            next_letter: (next_counts[next_letter] + smoothing) / (total_next + smoothing * V)
            for next_letter in all_letters
        }
    return start_prob, transition_prob

# -------------------------
# Train HMMs for each word length (same as yours)
# -------------------------
def train_length_based_hmms(words, smoothing=1.0):
    length_word_map = defaultdict(list)
    for word in words:
        length_word_map[len(word)].append(word)

    length_hmm_models = {}
    for length, words_of_length in sorted(length_word_map.items()):
        print(f"Training HMM for word length {length} with {len(words_of_length)} words")
        start_prob, transition_prob = train_hmm_for_words(words_of_length, smoothing=smoothing)
        length_hmm_models[length] = (start_prob, transition_prob)
    return length_hmm_models

# -------------------------
# Compute probability of a whole word under the bigram HMM
# P(word) = start_prob[w0] * prod_{i=0..n-2} transition[w_i][w_{i+1}]
# returns log-prob for numerical stability, and exp(logp) when needed
# -------------------------
def word_log_probability(word, start_prob, transition_prob):
    if len(word) == 0:
        return -math.inf
    # use log-probs for stability
    logp = math.log(start_prob.get(word[0], 1e-12))
    for i in range(len(word) - 1):
        a = word[i]
        b = word[i + 1]
        trans = transition_prob.get(a, None)
        if trans is None:
            # unseen prev-letter -> assume tiny prob
            logp += math.log(1e-12)
        else:
            logp += math.log(trans.get(b, 1e-12))
    return logp

# -------------------------
# Filter words matching pattern and avoiding guessed letters
# pattern: list or string like ['_','A','_']
# guessed_letters: set of letters guessed so far (both correct & incorrect)
# This version enforces that letters already revealed in pattern match.
# -------------------------
def filter_words_by_pattern(words, pattern, guessed_letters):
    filtered = []
    pat_len = len(pattern)
    for word in words:
        if len(word) != pat_len:
            continue
        match = True
        for wc, pc in zip(word, pattern):
            if pc != '_' and pc != wc:
                match = False
                break
            if pc == '_' and wc in guessed_letters:
                # don't allow a word that has an already-guessed letter in an unrevealed spot
                match = False
                break
        if match:
            filtered.append(word)
    return filtered


# -------------------------
# Calculate letter probs using the HMM as an oracle:
# Score each candidate (filtered) word by its HMM probability,
# then sum (weight) unique occurrences of letters in remaining spots.
# That returns P(letter | pattern, guessed_letters) approximated via word probabilities.
# -------------------------
def calculate_letter_probs_using_hmm(words_of_length, pattern, guessed_letters, hmm):
    # hmm: (start_prob, transition_prob)
    start_prob, transition_prob = hmm
    # filter words consistent with current pattern & guessed letters
    candidates = filter_words_by_pattern(words_of_length, pattern, guessed_letters)

    # if no candidates, fallback to simple positional frequency
    if not candidates:
        # fallback: count letters at blank positions from all words_of_length
        letter_counts = Counter()
        total = 0
        for w in words_of_length:
            for idx, ch in enumerate(w):
                if pattern[idx] == '_' and ch not in guessed_letters:
                    letter_counts[ch] += 1
                    total += 1
        if total == 0:
            return {chr(ord('A') + i): 0.0 for i in range(26)}
        return {chr(ord('A') + i): (letter_counts[chr(ord('A') + i)] / total) for i in range(26)}

    # compute log-probabilities for candidates
    log_probs = [word_log_probability(w, start_prob, transition_prob) for w in candidates]
    # normalize in log-space -> convert to probabilities
    max_logp = max(log_probs)
    probs = [math.exp(lp - max_logp) for lp in log_probs]  # subtract max for numerical stability
    # normalize
    s = sum(probs)
    if s == 0:
        # numeric fallback (rare)
        probs = [1.0 / len(probs)] * len(probs)
    else:
        probs = [p / s for p in probs]

    # weight letters: count each letter once per word (unique letters in that word that are in blank positions)
    letter_scores = Counter()
    for w, p in zip(candidates, probs):
        # only count letters at blank positions (and not already guessed)
        letters_here = set(ch for i, ch in enumerate(w) if pattern[i] == '_' and ch not in guessed_letters)
        for L in letters_here:
            letter_scores[L] += p

    # normalize to probability distribution over alphabet
    letters = [chr(ord('A') + i) for i in range(26)]
    total_score = sum(letter_scores.values())
    if total_score == 0:
        return {L: 0.0 for L in letters}
    return {L: (letter_scores[L] / total_score) for L in letters}

# -------------------------
# Example usage patch (replace your calculate loop with this snippet)
# -------------------------
if __name__ == "__main__":
    corpus_path = "corpus.txt"
    all_words = load_corpus(corpus_path)

    hmms = train_length_based_hmms(all_words, smoothing=1.0)

    word_length = 5
    words_of_length = [w for w in all_words if len(w) == word_length]

    pattern = ['_'] * word_length
    guessed_letters = set()

    guesses = ['A', 'E', 'N']
    for guess in guesses:
        guessed_letters.add(guess)
        # Use HMM to compute letter probabilities
        hmm = hmms[word_length]
        letter_probs = calculate_letter_probs_using_hmm(words_of_length, pattern, guessed_letters, hmm)

        print(f"After guessing '{guess}':")
        sorted_probs = sorted(letter_probs.items(), key=lambda x: x[1], reverse=True)
        for letter, prob in sorted_probs[:8]:
            print(f"  {letter}: {prob:.4f}")

        if guess == 'N':
            pattern[-1] = 'N'
        print("Updated pattern:", "".join(pattern))
        print()

Training HMM for word length 1 with 46 words
Training HMM for word length 2 with 84 words
Training HMM for word length 3 with 388 words
Training HMM for word length 4 with 1169 words
Training HMM for word length 5 with 2340 words
Training HMM for word length 6 with 3755 words
Training HMM for word length 7 with 5111 words
Training HMM for word length 8 with 6348 words
Training HMM for word length 9 with 6808 words
Training HMM for word length 10 with 6465 words
Training HMM for word length 11 with 5452 words
Training HMM for word length 12 with 4292 words
Training HMM for word length 13 with 3094 words
Training HMM for word length 14 with 2019 words
Training HMM for word length 15 with 1226 words
Training HMM for word length 16 with 698 words
Training HMM for word length 17 with 375 words
Training HMM for word length 18 with 174 words
Training HMM for word length 19 with 88 words
Training HMM for word length 20 with 40 words
Training HMM for word length 21 with 16 words
Training HMM fo

In [ ]:
# ==========================
# HANGMAN HYBRID RL (Q-LEARNING + HMM PRIORS)
# Optimized for Hackathon Evaluation
# ==========================

import random, math
from collections import defaultdict, Counter
from tqdm import tqdm
import numpy as np

# -------------------------
# ENVIRONMENT
# -------------------------
class HangmanEnv:
    def __init__(self, word, max_wrong=6):
        self.word = word
        self.word_set = set(word)
        self.max_wrong = max_wrong
        self.reset()

    def reset(self):
        self.guessed = set()
        self.wrong = 0
        self.done = False
        self.state = ['_' for _ in self.word]
        return self._get_obs()

    def _get_obs(self):
        # returns (pattern, wrong_count, length)
        return (''.join(self.state), self.wrong, len(self.word))

    def step(self, action):
        """Takes a letter as an action, returns next_state, reward, done, info."""
        if self.done:
            return self._get_obs(), 0, True, {}

        reward = 0
        if action in self.guessed:
            reward = -4  # repeated guess penalty
        else:
            self.guessed.add(action)
            if action in self.word_set:
                prev = self.state.copy()
                for i, c in enumerate(self.word):
                    if c == action:
                        self.state[i] = action
                new_reveals = sum(a != b for a, b in zip(self.state, prev))
                reward = 10 * new_reveals  # reward for new reveals
            else:
                self.wrong += 1
                reward = -7  # wrong guess penalty

        if '_' not in self.state:
            self.done = True
            reward += 80  # win bonus
        elif self.wrong >= self.max_wrong:
            self.done = True
            reward -= 40  # fail penalty

        return self._get_obs(), reward, self.done, {}

# -------------------------
# UTILITIES
# -------------------------
def load_words(path):
    with open(path, 'r') as f:
        return [w.strip().upper() for w in f if w.strip()]

def build_letter_priors(words):
    """Compute global unigram letter frequencies."""
    cnt = Counter()
    total = 0
    for w in words:
        cnt.update(w)
        total += len(w)
    return {c: cnt[c] / total for c in cnt}

def build_positional_priors(words):
    """Compute letter probabilities by position."""
    pos_counts = defaultdict(Counter)
    pos_totals = Counter()
    for w in words:
        for i, ch in enumerate(w):
            pos_counts[i][ch] += 1
            pos_totals[i] += 1
    pos_priors = {i: {ch: pos_counts[i][ch] / pos_totals[i] for ch in pos_counts[i]} for i in pos_counts}
    return pos_priors

# -------------------------
# HYBRID Q-LEARNING AGENT
# -------------------------
class HybridQLearningAgent:
    def __init__(self, hmm_priors, pos_priors, alpha=0.15, gamma=0.95, epsilon=0.3, epsilon_decay=0.9995, epsilon_min=0.05):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.actions = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
        self.hmm_priors = hmm_priors
        self.pos_priors = pos_priors

    def encode_state(self, obs):
        pattern, wrong, length = obs
        return pattern + f"|{wrong}|{length}"

    def get_action(self, obs, guessed):
        state = self.encode_state(obs)
        available = [a for a in self.actions if a not in guessed]
        if not available:
            return random.choice(self.actions)

        # epsilon-greedy with decay
        if random.random() < self.epsilon:
            return random.choice(available)

        # bias with HMM + positional priors
        pattern, _, _ = obs
        hmm_bias = {}
        for a in available:
            base = self.hmm_priors.get(a, 0)
            # add small boost if letter commonly appears in any blank position
            pos_bonus = np.mean([self.pos_priors.get(i, {}).get(a, 0) for i, p in enumerate(pattern) if p == '_']) if '_' in pattern else 0
            hmm_bias[a] = 0.3 * base + 0.7 * pos_bonus

        q_values = {a: self.q_table[state][a] + 0.5 * hmm_bias.get(a, 0) for a in available}
        return max(q_values, key=q_values.get)

    def update(self, obs, action, reward, next_obs):
        s = self.encode_state(obs)
        s_next = self.encode_state(next_obs)
        best_next = max(self.q_table[s_next].values(), default=0)
        self.q_table[s][action] += self.alpha * (reward + self.gamma * best_next - self.q_table[s][action])
        # decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# -------------------------
# TRAINING
# -------------------------
def train_agent(agent, train_words, episodes=20000):
    # curriculum: short words → long words
    for ep in tqdm(range(episodes)):
        if ep < 5000:
            word = random.choice([w for w in train_words if len(w) <= 5])
        elif ep < 15000:
            word = random.choice([w for w in train_words if len(w) <= 7])
        else:
            word = random.choice(train_words)

        env = HangmanEnv(word)
        obs = env.reset()
        while True:
            action = agent.get_action(obs, env.guessed)
            next_obs, reward, done, _ = env.step(action)
            agent.update(obs, action, reward, next_obs)
            obs = next_obs
            if done:
                break
    return agent # Added return statement

# -------------------------
# EVALUATION
# -------------------------
def evaluate_agent(agent, test_words):
    success = 0
    total_wrong = 0
    total_repeat = 0

    for word in test_words:
        env = HangmanEnv(word)
        obs = env.reset()
        while not env.done:
            action = agent.get_action(obs, env.guessed)
            if action in env.guessed:
                total_repeat += 1
            prev_wrong = env.wrong
            _, _, done, _ = env.step(action)
            if env.wrong > prev_wrong:
                total_wrong += 1
            obs = env._get_obs()
        if '_' not in env.state:
            success += 1

    success_rate = success / len(test_words)
    final_score = (success_rate * 2000) - (total_wrong * 5) - (total_repeat * 2)
    return success_rate, total_wrong, total_repeat, final_score

# -------------------------
# MAIN EXECUTION
# -------------------------
train_words = load_words("corpus.txt")
test_words = load_words("test.txt")

hmm_priors = build_letter_priors(train_words)
pos_priors = build_positional_priors(train_words)

agent = HybridQLearningAgent(
    hmm_priors=hmm_priors,
    pos_priors=pos_priors,
    alpha=0.15,
    gamma=0.95,
    epsilon=0.3,
    epsilon_decay=0.9995,
    epsilon_min=0.05
)

print("Training hybrid agent...")
train_agent(agent, train_words, episodes=20000)

print("\nEvaluating...")
sr, tw, tr, fs = evaluate_agent(agent, test_words)

print("\n===== FINAL RESULTS =====")
print(f"✅ Success Rate: {sr*100:.2f}%")
print(f"❌ Total Wrong Guesses: {tw}")
print(f"🔁 Total Repeated Guesses: {tr}")
print(f"🏆 Final Score: {fs:.2f}")

Training hybrid agent...


100%|██████████| 20000/20000 [01:22<00:00, 243.87it/s]



Evaluating...

===== FINAL RESULTS =====
✅ Success Rate: 14.85%
❌ Total Wrong Guesses: 11401
🔁 Total Repeated Guesses: 0
🏆 Final Score: -56708.00


In [ ]:
!pip install gradio numpy


In [ ]:
import gradio as gr
import numpy as np
from collections import defaultdict, Counter

# ----------------------------
# 1️⃣ HMM Training Functions
# ----------------------------
def load_corpus(filepath='/content/corpus.txt'):
    # Example fallback if no file given
    if filepath is None:
        words = ["apple", "book", "data", "cat", "dog", "banana", "paper", "people"]
    else:
        with open(filepath, 'r') as f:
            words = [line.strip().lower() for line in f if line.strip()]
    return words

def build_letter_priors(words):
    counts = Counter("".join(words))
    total = sum(counts.values())
    return {ch: c / total for ch, c in counts.items()}

def build_positional_priors(words):
    pos_priors = defaultdict(Counter)
    for w in words:
        for i, ch in enumerate(w):
            pos_priors[i][ch] += 1
    # Normalize
    for i in pos_priors:
        total = sum(pos_priors[i].values())
        for ch in pos_priors[i]:
            pos_priors[i][ch] /= total
    return pos_priors

# ----------------------------
# 2️⃣ Hybrid Q-Learning Agent
# ----------------------------
class HybridQLearningAgent:
    def __init__(self, hmm_priors, pos_priors, alpha=0.15, gamma=0.95, epsilon=0.3,
                 epsilon_decay=0.9995, epsilon_min=0.05):
        self.hmm_priors = hmm_priors
        self.pos_priors = pos_priors
        self.q_table = defaultdict(lambda: defaultdict(float))
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

    def get_action(self, obs, guessed):
        pattern, _, _ = obs
        available = [ch for ch in "abcdefghijklmnopqrstuvwxyz" if ch not in guessed]
        state = pattern

        # If no unknowns, return done
        if "_" not in pattern:
            return None

        # HMM + positional priors
        hmm_bias = {}
        for a in available:
            base = self.hmm_priors.get(a, 0)
            pos_bonus = np.mean([
                self.pos_priors.get(i, {}).get(a, 0)
                for i, p in enumerate(pattern)
                if p == '_'
            ]) if '_' in pattern else 0
            hmm_bias[a] = 0.3 * base + 0.7 * pos_bonus

        # Combine with Q-values
        q_values = {a: self.q_table[state][a] + 0.5 * hmm_bias.get(a, 0) for a in available}

        # Choose best letter
        best = max(q_values, key=q_values.get)
        return best

# ----------------------------
# 3️⃣ Initialize Data & Agent
# ----------------------------
words = load_corpus('/content/test.txt')
hmm_priors = build_letter_priors(words)
pos_priors = build_positional_priors(words)
agent = HybridQLearningAgent(hmm_priors, pos_priors)

# ----------------------------
# 4️⃣ Prediction Logic
# ----------------------------
def predict_word(masked_word):
    masked_word = masked_word.lower().strip()
    if not masked_word or '_' not in masked_word:
        return "Please enter a masked word like '_p__e'."

    guessed = set([ch for ch in masked_word if ch != '_'])
    obs = (masked_word, 0, 6)
    next_guess = agent.get_action(obs, guessed)

    if next_guess is None:
        return "Word seems complete!"
    return f"🤖 Suggested next letter: '{next_guess.upper()}'"

# ----------------------------
# 5️⃣ Gradio UI
# ----------------------------
with gr.Blocks(title="Hangman Word Predictor (HMM + RL)") as demo:
    gr.Markdown("## 🧩 Hangman Predictor (Hybrid HMM + Q-Learning)")
    gr.Markdown("Enter a masked word (use `_` for unknown letters). Example: `_p__e`")

    masked_input = gr.Textbox(label="Masked Word", placeholder="_p__e", max_lines=1)
    predicted_output = gr.Textbox(label="Model Suggestion")

    predict_btn = gr.Button("Predict Next Letter")
    predict_btn.click(fn=predict_word, inputs=masked_input, outputs=predicted_output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a073a8d333d31fb442.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
